# Zernikes

Prysm supports several flavors of Zernike polynomial:
- Fringe
- Noll
- ANSI "j"
- ANSI "n,m"

The single index notations have identical interfaces, while the (n,m) notation is slightly different.

Zernike notations are a subclass of [Pupil](./Pupils.ipynb), so they support the same arguments to `__init__`,

In [ ]:
%matplotlib inline
from prysm import FringeZernike, NollZernike, ANSI1TermZernike, ANSI2TermZernike
p = FringeZernike(samples=123, dia=456.7, wavelength=1.0, z_unit='nm', phase_mask='dodecagon')

There are additional keyword arguments for each term.  The polynomials are orthogonal, having unit amplitude (zero-to-peak).  They can also be used with unit variance via the `norm` keyword argument, in which case they are orthonormal.  The polynomial classes have friendly print statements:

In [ ]:
p2 = FringeZernike(Z4=1, Z9=1, Z98=1, norm=True)
print(p2)

Notice that we include a 98th term which you likely will not find in most other programs, and that the RMS value is equal to sqrt(1^2 + 1^2 + 1^2) = sqrt(3) = 1.718 ~= 1.722. The difference is due to the array sizes used by prysm by default, if increased, e.g. by adding `samples=1204` to the constructor, the value would converge to the analytical one.

A Zernike pupil can also be initalized with an iterable (list, tuple...) of coefficients,

In [ ]:
import numpy as np
terms = np.random.rand(49)  # 49 zernike coefficients, e.g. from a wavefront sensor
fz3 = FringeZernike(terms)

Zernike instances have a `truncate` method which discards terms with indices higher than n. For example,

In [ ]:
fz3.truncate(16)
print(fz3)

In [ ]:
fz3.top_n(5)

or the terms listed by their pairwise magnitudes and clocking angles,

In [ ]:
fz3.magnitudes

Magnitudes is a dict keyed by the name with values of `(mag, ang)`.  The angle is always zero if the term is rotationally invariant.

These things may be (bar) plotted;

In [ ]:
fz3.barplot(orientation='h', buffer=1, zorder=3)
fz3.barplot_magnitudes(orientation='h', buffer=1, zorder=3)
fz3.barplot_topn(n=5, orientation='h', buffer=1, zorder=3)

`orientation` controls the axis on which the terms are enumerated. `h` results in vertical bars, `v` is also accepted, as are horizontal and vertical. `buffer` is the number of terms’ worth of spaces left on each side. The default of 1 leaves a one bar margin. `zorder` is passed to matplotlib – the default of 3 places the bars above any gridlines, which is an aesthetic choice. Matplotlib has a general default of 1.

If you would like direct access to the underlying functions, the first few polynomials are provided as explicit functions and can be imported:

In [ ]:
from prysm.zernike import defocus

each of these takes arguments of (rho, phi). They have names which end with _x, or _00 and _45 for the terms which have that naming convention.

Zernike functions are cached by prysm's internal routines,

In [ ]:
from prysm.zernike import zcachemn
# 8 is the index into prysm.zernike.zernikes, which loosely follows the Fringe ordering
zcachemn(7, 7, norm=True, samples=128)
zcachemn.clear()  # you should never have to do this unless you want to release memory

This cache instance is used internally by prysm, if you modify the returned arrays in-place, you probably won’t like the result. You can create your own independent instance,

In [ ]:
from prysm.zernike import ZCacheMN
my_zcache = ZCacheMN()

See [Pupils](./Pupils.ipynb) for information about general pupil functions.